# Incoherent Scatter Blackbox Existing Radar Simulator

This jupyter notebook runs a gui interface to simulate an existing ISR radar.  It requires ipywidgets be installed, and the ISR_blackbox module.  The ISR_blackbox code is available in this directory, and <a href="ISR_Blackbox.py">here</a>.
    
Written by Bill Rideout - brideout@mit.edu    

In [1]:
import os, sys, os.path
import IPython.display

sys.path.append('.')
import ISR_blackbox
import ipywidgets

These next cells create the ipywidgets used in the GUI.

In [2]:
title_label = ipywidgets.HTML(value='<h3><center>Existing ISR Blackbox Performance Predictor</center></h3>')
isr_service = ISR_blackbox.ISR_Blackbox_Service()
names = ['Please choose an ISR radar'] + isr_service.getRadarNames()
dropdown_radars = ipywidgets.Dropdown(options=names)

In [3]:
modes = []
dropdown_modes = ipywidgets.Dropdown(options=modes)
modes_label = ipywidgets.Label(value='Choose mode:')

In [4]:
months_label = ipywidgets.Label(value='Choose start month and day: ')
months = range(1,13)
dropdown_months = ipywidgets.Dropdown(options=months, description='Month:')

In [5]:
days = range(1,32)
dropdown_days = ipywidgets.Dropdown(options=days, description='Day:')

In [6]:
hours_label = ipywidgets.Label(value='Choose UT start hour and hours to run: ')
hours = range(24)
dropdown_hours = ipywidgets.Dropdown(options=hours, description='Start hour:')
run_hours = [3,6,12,24]
dropdown_run = ipywidgets.Dropdown(options=run_hours, description='Run hours:')

In [7]:
elevation_set = ipywidgets.BoundedFloatText(min=0, max=90, value=90, step=1, description='Elevation:')
integration_label = ipywidgets.Label(value='Integration time: ')
integration_set = ipywidgets.BoundedFloatText(min=0, max=3600, value=30, step=1, description='Seconds:')
button = ipywidgets.Button(description='Calculate ISR Performance',
                           layout=ipywidgets.Layout(height='auto', width='auto'))

These next cells create the event handlers for the GUI, and create a function plot_existing_acf that calls the ISR_blackbox module once the inputs are gathered to create a plot.

In [8]:
plot_output = ipywidgets.Output()

def dropdown_radars_eventhandler(change):
    modes = isr_service.getModeNames(change.new)
    dropdown_modes.options = modes
    plot_output.clear_output()
    
dropdown_radars.observe(dropdown_radars_eventhandler, names='value')

In [9]:
def plot_existing_acf(radar_name, mode_name, start_month, start_day, start_hour, total_hours, el, integration_seconds):
    plot_output.clear_output()
    with plot_output:
        existingISR = ISR_blackbox.Existing_ISR_Radars()
        existingISR.plot_acf_err_ratio(radar_name, mode_name, None, start_month, start_day, start_hour, total_hours, el, integration_seconds)

In [10]:
def button_eventhandler(obj):
    radar_name = dropdown_radars.value
    if radar_name.find('Please choose') != -1:
        plot_output.clear_output()
        with plot_output:
            display('YOU MUST FIRST CHOOSE AN EXISTING RADAR AND MODE')
            return
    mode_name = dropdown_modes.value
    start_month = int(dropdown_months.value)
    start_day = int(dropdown_days.value)
    start_hour = int(dropdown_hours.value)
    total_hours = int(dropdown_run.value)
    el = int(elevation_set.value)
    integration_seconds = int(integration_set.value)
    plot_existing_acf(radar_name, mode_name,start_month, start_day, start_hour, total_hours, el, integration_seconds)
    
button.on_click(button_eventhandler)

In [11]:
row11_widgets = ipywidgets.HBox([modes_label, dropdown_modes])
row21_widgets = ipywidgets.HBox([dropdown_months, dropdown_days])
row31_widgets = ipywidgets.HBox([dropdown_hours, dropdown_run])
row41_widgets = ipywidgets.HBox([integration_label, integration_set])

Finally, display the GUI:

In [12]:
grid = ipywidgets.GridspecLayout(12,2)
grid[0,:] = title_label
grid[1,0] = dropdown_radars
grid[1,1] = row11_widgets
grid[2,0] = months_label
grid[2,1] = row21_widgets
grid[3,0] = hours_label
grid[3,1] = row31_widgets
grid[4,0] = elevation_set
grid[4,1] = row41_widgets
grid[5,:] = button
grid[6:11,:] = plot_output
grid

GridspecLayout(children=(HTML(value='<h3><center>Existing ISR Blackbox Performance Predictor</center></h3>', l…